In [13]:
import tensorflow as tf
from scipy.io import loadmat
import numpy as np
import matplotlib.pyplot as plt
import datetime


In [14]:
mat = loadmat("f1data.mat")

x    = mat['x'][:,0]
y    = mat['y'][:,0]
tail = mat['tail'][:-1,0]

input_data = np.stack((x,y)).T
output_data = np.stack((x,y)).T


In [15]:
WINDOW_SIZE = 500
OUTPUT_SIZE = 50
BATCH_SIZE  = 500

# Tail To Pos

In [16]:
class TailToPosGenerator(tf.keras.utils.Sequence):
   
    def __init__(self, input_data,output_data , input_window_size , output_window_size,batch_size):
        self.batch_size = batch_size
        self.input_data = input_data
        self.output_data = output_data
        self.window_size = input_window_size
        self.output_size = output_window_size
        self.EPOCH_LENGHT = 1000
        self.on_epoch_end()
        
        self.x = np.zeros((self.batch_size,self.window_size,2),dtype=np.float32)
        self.y = np.zeros((self.batch_size,self.output_size,2),dtype=np.float32)
      
    def __len__(self):
        return self.EPOCH_LENGHT

    def __getitem__(self, index):
        # [batch, timesteps, feature]

        idx = self.index[index]

        for i in range(idx,idx+self.batch_size):
            in_start  = i
            in_stop   = in_start + self.window_size
            out_start = in_stop
            out_stop  = out_start + self.output_size

            self.x[i-idx,:,:] = self.input_data[in_start:in_stop]
            self.y[i-idx,:,:] = self.output_data[out_start:out_stop,:]
            
        return self.x, self.y

    def on_epoch_end(self):
        self.index = idx = np.random.randint(low=0,high=self.input_data.shape[0] - (self.batch_size + self.window_size + self.output_size)  , size=self.EPOCH_LENGHT)


In [17]:
def TailToPosLSTMModel(window_size,output_size):
    model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(window_size,2),name='inpt'),
    tf.keras.layers.LSTM(64, return_sequences=False,name='lstm'),
    tf.keras.layers.Dense(output_size*2,kernel_initializer=tf.initializers.zeros,name='dense'),
    tf.keras.layers.Reshape([output_size, 2],name='reshape')
    ],name='full_net')

    model.compile(
                #loss=EuclideanDistanceLoss,
                loss=tf.losses.MeanSquaredError(),
                optimizer=tf.optimizers.Adam(),
                metrics=[tf.metrics.MeanAbsoluteError()])

    model.summary()

    return model

In [18]:
def EuclideanDistanceLoss(y_true, y_pred):
    cumsum_y_true = tf.math.cumsum(y_true, axis=1)
    cumsum_y_pred = tf.math.cumsum(y_pred, axis=1)

    return tf.norm(cumsum_y_true-cumsum_y_pred, ord='euclidean')

In [ ]:
model = TailToPosLSTMModel( window_size = WINDOW_SIZE, output_size = OUTPUT_SIZE)

gen =TailToPosGenerator(input_data,output_data,WINDOW_SIZE,OUTPUT_SIZE,BATCH_SIZE)

model.fit(gen,epochs= 1000,verbose=1)

Model: "full_net"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 64)                17152     
_________________________________________________________________
dense (Dense)                (None, 100)               6500      
_________________________________________________________________
reshape (Reshape)            (None, 50, 2)             0         
Total params: 23,652
Trainable params: 23,652
Non-trainable params: 0
_________________________________________________________________
Epoch 1/1000
1000/1000 [==============================] - 33s 33ms/step - loss: 0.0587 - mean_absolute_error: 0.1154
Epoch 2/1000
1000/1000 [==============================] - 33s 33ms/step - loss: 0.0560 - mean_absolute_error: 0.1128
Epoch 3/1000
1000/1000 [==============================] - 33s 33ms/step - loss: 0.0522 - mean_absolute_error: 0.1111
Epoch 4/1000
1000/1000 [=========

# Plots

In [ ]:
t = np.zeros((1,WINDOW_SIZE,3),dtype=np.float32)
ty = np.zeros((1,OUTPUT_SIZE,3),dtype=np.float32)

start  = 134
t[0,:,:] = data[start:start+WINDOW_SIZE]
ty[0,:,:] = data[start+WINDOW_SIZE:start+WINDOW_SIZE+OUTPUT_SIZE]

y  = model(t)

print(t.shape,y.shape)

In [ ]:
x = np.asarray(range(window_size))
xx = np.asarray(range(window_size,window_size + output_size))


N = data.shape[1]
for i in range(N):
    plt.figure(figsize=(30,2))
    plt.suptitle(data.columns[i], fontsize=20)
    plt.plot(x,t[0,:,i])
    plt.plot(xx,y[0,:,i],color='green')
    plt.plot(xx,ty[0,:,i],color='red')
plt.show()

In [ ]:
x = np.asarray(range(WINDOW_SIZE))
xx = np.asarray(range(WINDOW_SIZE,WINDOW_SIZE + OUTPUT_SIZE))


for i in range(2):
    plt.figure(figsize=(30,2))
    plt.plot(x,  t[0,:,i])
    plt.plot(xx,ty[0,:,i],color='green')
    plt.plot(xx, y[0,:,i],color='red')
    plt.show()




In [ ]:
prevx = np.cumsum(np.squeeze(t[0,:,0]))
prevy = np.cumsum(np.squeeze(t[0,:,1]))

trux = prevx[-1] + np.cumsum(np.squeeze(ty[0,:,0]))
truy = prevy[-1] + np.cumsum(np.squeeze(ty[0,:,1]))

predx = prevx[-1] + np.cumsum(np.squeeze())
predy = prevy[-1] + np.cumsum(np.squeeze(y[0,:,1]))


plt.plot(prevx,prevy,color='blue')
plt.plot(trux,truy,color='green')
# plt.plot(predx,predy,color='red')


plt.show()

In [17]:
gen =TailToPosGenerator(input_data,output_data,10,5,3)

x,y = gen[0]


# print('x',x,x.shape)
# print('y',y,y.shape)




print(y)
print(cm)


l2_norm = tf.norm(y-y*5, ord='euclidean')

print(l2_norm)

[[[ 0.05200195 -0.02529907]
  [ 0.02697754 -0.04050446]
  [-0.05187988  0.02540588]
  [ 0.05187988 -0.02540588]
  [ 0.          0.        ]]

 [[ 0.02697754 -0.04050446]
  [-0.05187988  0.02540588]
  [ 0.05187988 -0.02540588]
  [ 0.          0.        ]
  [ 0.          0.        ]]

 [[-0.05187988  0.02540588]
  [ 0.05187988 -0.02540588]
  [ 0.          0.        ]
  [ 0.          0.        ]
  [ 0.04351807 -0.01239777]]]
tf.Tensor(
[[[ 0.05200195 -0.02529907]
  [ 0.07897949 -0.06580353]
  [ 0.02709961 -0.04039764]
  [ 0.07897949 -0.06580353]
  [ 0.07897949 -0.06580353]]

 [[ 0.02697754 -0.04050446]
  [-0.02490234 -0.01509857]
  [ 0.02697754 -0.04050446]
  [ 0.02697754 -0.04050446]
  [ 0.02697754 -0.04050446]]

 [[-0.05187988  0.02540588]
  [ 0.          0.        ]
  [ 0.          0.        ]
  [ 0.          0.        ]
  [ 0.04351807 -0.01239777]]], shape=(3, 5, 2), dtype=float32)
tf.Tensor(0.6945555, shape=(), dtype=float32)
